In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import transformers
import load_data
import torch
from torch import nn
from transformers import BertModel, BertTokenizer, CamembertModel, CamembertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
import numpy as np
import os
from utils import *
from training_audio_model import *
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu') # GPU ou CPU
print(torch.cuda.is_available())
# Chargement des données
transcr_path='paco-cheese/transcr'
data=load_data.load_all_ipus(folder_path=transcr_path,load_words=False)

data[:30]

C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True


C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


,ipu_id,speaker,start,stop,text,is_main_speaker,turn_at_start,turn_after,turn_start_word,yield_at_end,request_at_start,dyad
0,0,AA,4.54,4.840,tu as,True,False,True,4.84,True,False,transcr\AAOR
1,1,OR,5.14,5.825,mh ouais si tu veux,True,True,False,NaN,False,True,transcr\AAOR
2,2,OR,6.62,7.010,frog joke,True,False,False,NaN,False,False,transcr\AAOR
3,3,OR,7.42,10.870,un jour un ingénieur traversait la rue quand u...,True,False,False,NaN,False,False,transcr\AAOR
4,4,OR,11.36,13.835,si tu m'embrasses je me transforme en belle pr...,True,False,False,NaN,False,False,transcr\AAOR
5,5,OR,14.42,17.243,il s'accroupit ramassa la grenouille et la mit...,True,False,False,NaN,False,False,transcr\AAOR
6,6,OR,17.68,19.815,la grenouille reprit la parole et dit si tu m'...,True,False,False,NaN,False,False,transcr\AAOR
7,7,OR,20.04,21.230,et me tranformes en belle princesse,True,False,False,NaN,False,False,transcr\AAOR
8,8,OR,21.54,22.665,je resterai avec toi une semaine,True,False,False,NaN,False,False,transcr\AAOR
9,9,OR,23.50,25.680,l'ingénieur sortit la grenouille de sa poche,True,False,False,NaN,False,False,transcr\AAOR


In [2]:

df = pd.DataFrame(data)

print("y de base")
y=create_y(df) # Création de la target
print(y.count(1))
print(y.count(0))


print("y yield at")
y=create_y_yield_at(df) # Création de la target
print(y.count(1))
print(y.count(0))


"""
print("y turn after")
y=create_y_turn_after(df) # Création de la target
print(y.count(1))
print(y.count(0))
print(len(y))
print(len(data))
"""

y de base
9131
7269
y yield at
3090
13310


'\nprint("y turn after")\ny=create_y_turn_after(df) # Création de la target\nprint(y.count(1))\nprint(y.count(0))\nprint(len(y))\nprint(len(data))\n'

In [3]:

audio_files_path = 'paco-cheese/audio/2_channels/'


audio_segments = extract_audio_segments(data,audio_files_path)
audio_segments

 ...]

In [4]:
print(len(audio_segments))
print(len(y))


16400
16400


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier


limit=len(y)
X = np.array([extract_features(segment) for segment in audio_segments[:limit]])
x_save=X


C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1920
  warnings.warn(
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1280
  warnings.warn(
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1120
  warnings.warn(
C:\Users\victo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=18

In [6]:
"""class AudioDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        audio_sequence = torch.from_numpy(self.data[idx]).float()
        label = self.labels[idx]
        return audio_sequence, label
"""
dataset = AudioDataset(X, y)
train_size=int(0.8*len(dataset))
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

# Fixez le seed pour la reproductibilité
seed = 42
#limit=50000
X=X[:limit]
y=y[:limit]
print(len(X))
# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

# Créez les objets AudioDataset pour l'entraînement et le test
train_dataset = AudioDataset(X_train, y_train)
test_dataset = AudioDataset(X_test, y_test)

# Créez les DataLoaders pour ces ensembles de données
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



16400


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)
from training_audio_model import *
model = AudioCNN().to(device)

"""
class AudioRNN(nn.Module):
    def __init__(self, input_size=13, hidden_size=64, num_layers=1):
        super(AudioRNN, self).__init__()
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        
        # Assuming output is pooled to a fixed size
        rnn_output_size = hidden_size  
        self.fc1 = nn.Linear(rnn_output_size, 500)
        self.fc2 = nn.Linear(500, 100)
        self.fc3 = nn.Linear(100, 2)

    def forward(self, x):
        # x shape: [batch, seq_len, features]
        x, _ = self.rnn(x)
        #x = x[:, -1, :]  # Taking the last output

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
"""
#model=AudioRNN(input_size=13,hidden_size=64).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Paramètres d'entraînement
num_epochs = 10
class_weights = torch.tensor([1.0, 4.0], device=device)

loss_fn = nn.CrossEntropyLoss(weight=class_weights)

#entraînement
for epoch in range(num_epochs):

    model.train()
    
    for inputs, labels in train_loader:
    

   
        inputs, labels = inputs.to(device), labels.to(device)
       

        optimizer.zero_grad()


        outputs = model(inputs)
        loss = loss_fn(outputs, labels)


        loss.backward()
        optimizer.step()

    # Évaluation
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
          
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {100 * correct / total:.2f}%')
    torch.save(model, 'modele/model_audio')

Using device: cuda
Epoch [1/10], Loss: 0.6821, Accuracy: 48.66%
Epoch [2/10], Loss: 0.6912, Accuracy: 81.37%
Epoch [3/10], Loss: 0.6060, Accuracy: 81.46%
Epoch [4/10], Loss: 0.6656, Accuracy: 39.82%
Epoch [5/10], Loss: 0.6796, Accuracy: 46.28%
Epoch [6/10], Loss: 0.6838, Accuracy: 58.02%
Epoch [7/10], Loss: 0.7299, Accuracy: 79.21%
Epoch [8/10], Loss: 0.6546, Accuracy: 45.61%
Epoch [9/10], Loss: 0.6287, Accuracy: 64.05%
Epoch [10/10], Loss: 0.6383, Accuracy: 66.01%


In [8]:

all_preds_audio,all_labels=predition_model_audio(model,test_loader,device,proba=False)
print(all_preds_audio[:40])
print(all_labels[:40])
f1 = f1_score(all_labels, all_preds_audio)#, average='weighted')
conf_matrix = confusion_matrix(all_labels, all_preds_audio)
print(f'Test F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')

total_class_0 = np.sum(conf_matrix[0])
total_class_1 = np.sum(conf_matrix[1])
detected_class_0 = conf_matrix[0, 0]  # Vrais positifs pour la classe 0
detected_class_1 = conf_matrix[1, 1]  # Vrais positifs pour la classe 1

print(f'Nombre d\'éléments de classe 0 détectés : {detected_class_0} sur {total_class_0}')
print(f'Nombre d\'éléments de classe 1 détectés : {detected_class_1} sur {total_class_1}')

[0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0
 1 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0
 0 0 0]
Test F1 Score: 0.2811089619600258
Confusion Matrix:
[[1947  725]
 [ 390  218]]
Nombre d'éléments de classe 0 détectés : 1947 sur 2672
Nombre d'éléments de classe 1 détectés : 218 sur 608
